In [15]:
import pandas as pd
import numpy as np


In [2]:
path_to_firstdf = '/Users/Artem_Boltaev/Documents/EPAM Projects/6. RecSys_course/source_code/recsys_course_epam/data/raw/recsys_task0_dataset.parquet'

df = pd.read_parquet(path_to_firstdf, engine='pyarrow')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1188 entries, 1 to 6040
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ground_truth  1188 non-null   object
 1   prediction    1188 non-null   object
dtypes: object(2)
memory usage: 27.8+ KB


In [4]:
df.head()

,ground_truth,prediction
user_id,,
1,"[1193, 3408, 2355, 1287, 2804, 594, 919, 595, ...","[2858, 260, 1196, 1198, 593, 2028, 318, 527, 2..."
2,"[1357, 3068, 1537, 2194, 648, 2268, 3468, 1210...","[2858, 260, 1196, 1198, 593, 2028, 318, 527, 2..."
3,"[3421, 1394, 104, 2735, 1210, 1079, 1615, 1291...","[2858, 260, 1196, 1198, 593, 2028, 318, 527, 2..."
4,"[3468, 2951, 1214, 1036, 260, 2028, 480, 1198,...","[2858, 260, 1196, 1198, 593, 2028, 318, 527, 2..."
5,"[2987, 2333, 1175, 2337, 1535, 1392, 866, 2770...","[2858, 260, 1196, 1198, 593, 2028, 318, 527, 2..."


In [ ]:
y_true = df['ground_truth'].apply(pd.Series).to_numpy()
y_pred = df['prediction'].apply(pd.Series).to_numpy()

# 1.HitRate@k

In [24]:

def hit_rate_at_k_score(y_true, y_pred, k: int):
    """
    Calculate Hit Rate at k.

    Parameters
    ----------
    y_true : array-like of shape (n_samples, n_items)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples, n_items)
        Estimated target values.

    k : int, default=6
        Number of recommendations to take into account.

    Returns
    -------
    score : float
        A non-negative floating point value (the best value is 1.0).
    """
    
    hits = [np.in1d(pred[:k], fact).any() for fact, pred in zip(y_true, y_pred)]
    return np.mean(hits)

Expected scores:

HitRate@3: 0.418

HitRate@5: 0.492

In [26]:
hit_rate_at_k_score(y_true, y_pred, 3)

0.4175084175084175

In [27]:
hit_rate_at_k_score(y_true, y_pred, 5)

0.49158249158249157

# 2.MAP@k

In [28]:
def mean_average_precision_at_k_score(y_true, y_pred, k):
    """
    Calculate mean average precision at k (MAP@k).

    Parameters
    ----------
    y_true : array-like of shape (n_samples, n_items)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples, n_items)
        Estimated target values.

    k : int, default=6
        Maximum number of recommendations to take into account.

    Returns
    -------
    score : float
        A non-negative floating point value (the best value is 1.0).
    """

    def _average_precision_at_k_score(fact, pred):
        hits = np.in1d(pred[:k], fact)
        precision_at_k_list = [hits[:i].sum() / i for i in range(1, len(hits) + 1) if hits[i - 1]]
        if precision_at_k_list:
            return np.mean(precision_at_k_list)
        else:
            return 0.0
    
    point_scores_list = [_average_precision_at_k_score(fact, pred) for fact, pred in zip(y_true, y_pred)]
    return np.average(point_scores_list)

EXPECTED VALUES

MAP@3: 0.325

MAP@5: 0.333

In [29]:
mean_average_precision_at_k_score(y_true, y_pred, 3)

0.3247053872053872

In [30]:
mean_average_precision_at_k_score(y_true, y_pred, 5)

0.3326155069210625

# 3.NDCG@k

In [41]:
#NDCG@k
#https://gist.github.com/mblondel/7337391

def dcg_score(y_true, y_pred, k, gains="exponential"):
    """Discounted cumulative gain (DCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_pred : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    gains : str
        Whether gains should be "exponential" (default) or "linear".
    Returns
    -------
    DCG @k : float
    """
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    if gains == "exponential":
        gains = 2 ** y_true - 1
    elif gains == "linear":
        gains = y_true
    else:
        raise ValueError("Invalid gains option.")

    # highest rank is 1 so +2 instead of +1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_pred, k, gains="exponential"):
    """Normalized discounted cumulative gain (NDCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_pred : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    gains : str
        Whether gains should be "exponential" (default) or "linear".
    Returns
    -------
    NDCG @k : float
    """
    best = dcg_score(y_true, y_true, k, gains)
    actual = dcg_score(y_true, y_pred, k, gains)
    return actual / best

EXPECTED Values

NDCG@3: 0.238

NDCG@5: 0.223

In [42]:
ndcg_score(y_true[0], y_pred[0], 3, gains="exponential")

/var/folders/mf/bfrczh690hggqb7bqhybgv040000gp/T/ipykernel_53670/907325949.py:24: RuntimeWarning: overflow encountered in power
  gains = 2 ** y_true - 1


nan

In [43]:
ndcg_score(y_true[0], y_pred[0], 5, gains="exponential")

/var/folders/mf/bfrczh690hggqb7bqhybgv040000gp/T/ipykernel_53670/907325949.py:24: RuntimeWarning: overflow encountered in power
  gains = 2 ** y_true - 1


nan

In [37]:
y_true[0]

array([1.193e+03, 3.408e+03, 2.355e+03, 1.287e+03, 2.804e+03, 5.940e+02,
       9.190e+02, 5.950e+02, 9.380e+02, 2.398e+03, 2.918e+03, 1.035e+03,
       2.791e+03, 2.018e+03, 3.105e+03, 2.797e+03, 1.270e+03, 5.270e+02,
       4.800e+01, 1.097e+03, 1.721e+03, 1.545e+03, 2.294e+03, 3.186e+03,
       1.566e+03, 5.880e+02, 1.907e+03, 7.830e+02, 1.836e+03, 1.022e+03,
       2.762e+03, 1.500e+02, 1.000e+00, 1.961e+03, 1.962e+03, 2.692e+03,
       2.600e+02, 1.028e+03, 1.029e+03, 1.207e+03, 2.028e+03, 5.310e+02,
       3.114e+03, 6.080e+02, 1.246e+03,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan, 